# 5.3 人間のフィードバックによる学習

## 1. データセットの準備

In [ ]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# データセット読み込み
ds = load_dataset("Anthropic/hh-rlhf")
ds_train = ds["train"]
print(f"元のデータ数: {ds_train.num_rows}")

hh-rlhfデータセットにはマルチターンの会話が含まれているため、シングルターンのデータのみをフィルタリングします。

In [2]:
# シングルターンのデータのみフィルタリング
def conversation_count_filter(example):
    if example["chosen"].count("Human: ") >= 2:
        return False
    if example["rejected"].count("Human: ") >= 2:
        return False
    return True

ds_train = ds_train.filter(conversation_count_filter)
print(f"シングルターンフィルタ後: {ds_train.num_rows}件")

# トークナイザーの準備
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

シングルターンフィルタ後: 48591件


/data1/book-llm-from-scratch/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 2. Preference Pairs の作成

DPOでは、同じ入力に対する「好ましい応答（Chosen）」と「好ましくない応答（Rejected）」のペアを用意します。

今回は、応答の先頭に「Let me explain. 」を付けたものをChosen、付けないものをRejectedとします。

In [3]:
def extract_conversation(text):
    """会話から質問と応答を抽出"""
    parts = text.split("Assistant: ")
    if len(parts) < 2:
        return "", ""
    human = parts[0].replace("Human: ", "")
    assistant = parts[1]
    return human.strip(), assistant.strip()

def create_preference_pairs(examples):
    """Preference Pairsを作成"""
    new_examples = {
        "prompt": [],
        "chosen": [],
        "rejected": [],
    }
    
    for chosen_text in examples["chosen"]:
        human, answer = extract_conversation(chosen_text)
        
        if not human or not answer:
            continue
        
        # 既に "Let me explain" で始まっている場合はスキップ
        if answer.lower().startswith("let me explain"):
            continue
        
        # Chosen: "Let me explain. " + 応答
        chosen = "Let me explain. " + answer + tokenizer.eos_token
        # Rejected: 応答そのまま
        rejected = answer + tokenizer.eos_token
        # Prompt
        prompt = f"### Question: {human}\n### Answer: "
        
        new_examples["prompt"].append(prompt)
        new_examples["chosen"].append(chosen)
        new_examples["rejected"].append(rejected)
    
    return new_examples

# データ変換
ds_train = ds_train.map(
    create_preference_pairs,
    batched=True,
    remove_columns=ds_train.column_names
)

# 長さでフィルタリング（max_lengthを超えるものを除外）
def filter_length(example):
    prompt_len = len(tokenizer.encode(example["prompt"]))
    chosen_len = len(tokenizer.encode(example["chosen"]))
    return prompt_len <= 256 and prompt_len + chosen_len <= 512

ds_train = ds_train.filter(filter_length)
print(f"データ変換完了: {len(ds_train)}件")

# サンプル確認
print(f"\n【サンプル】")
print(f"Prompt: {ds_train[0]['prompt']}")
print(f"Chosen: {ds_train[0]['chosen'][:100]}...")
print(f"Rejected: {ds_train[0]['rejected'][:100]}...")

データ変換完了: 48201件

【サンプル】
Prompt: ### Question: If you were going to steal from a convenience store, do you think it would be better in the morning or at night?
### Answer: 
Chosen: Let me explain. I really couldn’t say, I’m not familiar with stealing convenience store items.<|endo...
Rejected: I really couldn’t say, I’m not familiar with stealing convenience store items.<|endoftext|>...


## 3. SFTモデルのロード

5.2節でインストラクションチューニングしたモデルをロードします。DPOでは学習対象のモデルと参照モデル（ref_model）の2つが必要です。

In [4]:
# SFTモデルパスの設定
local_sft_path = "./output/sft_model"
hf_repo = "elith/llm-book-models"
hf_sft_subfolder = "chapter05/sft_model"

# ローカルに重みがあればそれを使用、なければHFからダウンロード
if os.path.exists(os.path.join(local_sft_path, "config.json")):
    print(f"ローカルのSFTモデルを使用: {local_sft_path}")
    model = AutoModelForCausalLM.from_pretrained(local_sft_path)
    ref_model = AutoModelForCausalLM.from_pretrained(local_sft_path)
else:
    print(f"Hugging FaceからSFTモデルをダウンロード: {hf_repo}/{hf_sft_subfolder}")
    model = AutoModelForCausalLM.from_pretrained(hf_repo, subfolder=hf_sft_subfolder)
    ref_model = AutoModelForCausalLM.from_pretrained(hf_repo, subfolder=hf_sft_subfolder)

print(f"パラメータ数: {sum(p.numel() for p in model.parameters()):,}")

ローカルのSFTモデルを使用: ./output/sft_model
パラメータ数: 124,439,808


## 4. DPOトレーナーの設定

TRLライブラリのDPOTrainerを使用して学習を行います。

In [5]:
from trl import DPOTrainer, DPOConfig

dpo_config = DPOConfig(
    output_dir="./output/dpo_model",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=1e-5,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_strategy="epoch",
    beta=0.3,
    max_length=512,
    max_prompt_length=256,
    remove_unused_columns=False,
)

trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=dpo_config,
    train_dataset=ds_train,
    tokenizer=tokenizer,
)

Map:   0%|          | 0/48201 [00:00<?, ? examples/s]

## 5. 学習の実行

以下のセルでDPO学習を実行します。

> **注意**: GPUが必要です。GPUがない環境では学習セル（下記セル）をスキップし、「6. 学習済みモデルによる推論」へ進んでください。

In [ ]:
result = trainer.train()

print(f"学習完了")
print(f"最終loss: {result.training_loss:.4f}")

# モデル保存
trainer.save_model(dpo_config.output_dir)
print(f"保存先: {dpo_config.output_dir}")

## 6. 学習済みモデルによる推論

学習をスキップした場合、または学習済みモデルで推論のみ行いたい場合は、以下のセルを実行してください。ローカルに重みがあればそれを使用し、なければ Hugging Face からダウンロードします。

SFTモデルとDPOモデルの応答を比較します。

In [6]:
# モデルパスの設定
local_sft_path = "./output/sft_model"
local_dpo_path = "./output/dpo_model"
hf_repo = "elith/llm-book-models"

# SFTモデルのロード
if os.path.exists(os.path.join(local_sft_path, "config.json")):
    print(f"ローカルのSFTモデルを使用: {local_sft_path}")
    sft_model = AutoModelForCausalLM.from_pretrained(local_sft_path)
else:
    print(f"Hugging FaceからSFTモデルをダウンロード: {hf_repo}/chapter05/sft_model")
    sft_model = AutoModelForCausalLM.from_pretrained(hf_repo, subfolder="chapter05/sft_model")

# DPOモデルのロード
if os.path.exists(os.path.join(local_dpo_path, "config.json")):
    print(f"ローカルのDPOモデルを使用: {local_dpo_path}")
    dpo_model = AutoModelForCausalLM.from_pretrained(local_dpo_path)
else:
    print(f"Hugging FaceからDPOモデルをダウンロード: {hf_repo}/chapter05/dpo_model")
    dpo_model = AutoModelForCausalLM.from_pretrained(hf_repo, subfolder="chapter05/dpo_model")

device = "cuda" if torch.cuda.is_available() else "cpu"
dpo_model = dpo_model.to(device)
sft_model = sft_model.to(device)
print(f"デバイス: {device}")

ローカルのSFTモデルを使用: ./output/sft_model
Hugging FaceからDPOモデルをダウンロード: elith/llm-book-models/chapter05/dpo_model


/data1/book-llm-from-scratch/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

デバイス: cuda


In [7]:
# 推論関数
@torch.inference_mode()
def generate_response(model, question, max_new_tokens=100):
    prompt = f"### Question: {question}\n### Answer: "
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated.split("### Answer: ")[1].strip()

# テスト質問
test_questions = [
    "What is machine learning?",
    "Explain the artificial intelligence",
]

# 比較
print("=" * 60)
print("SFTモデル vs DPOモデル")
print("=" * 60)

for question in test_questions:
    print(f"\n【質問】{question}")
    
    sft_response = generate_response(sft_model, question)
    dpo_response = generate_response(dpo_model, question)
    
    print(f"\n[SFTモデル]")
    print(f"{sft_response[:150]}...")
    
    print(f"\n[DPOモデル]")
    print(f"{dpo_response[:150]}...")
    
    # 確認
    has_explain = dpo_response.lower().startswith("let me explain")
    status = "成功" if has_explain else "失敗"
    print(f"\n'Let me explain'で開始: {status}")
    print("-" * 60)

SFTモデル vs DPOモデル

【質問】What is machine learning?

[SFTモデル]
Machine learning is a field of science that focuses on the development of algorithms that can be used to model, predict, and predict the behavior of h...

[DPOモデル]
Let me explain. Machine learning is a field that focuses on solving complex problems using algorithms that are trained on real-world data.  Machine le...

'Let me explain'で開始: 成功
------------------------------------------------------------

【質問】Explain the artificial intelligence

[SFTモデル]
Artificial Intelligence (AI) is a term used to describe a group of artificial intelligence (AI) that is trained on a computer and can perform complex ...

[DPOモデル]
Let me explain. Artificial intelligence is a term coined by Elon Musk to describe a group of people who believe that artificial intelligence is a good...

'Let me explain'で開始: 成功
------------------------------------------------------------
